In [1]:
import numpy as np
import matplotlib.animation as animation
import matplotlib.pyplot as plt
%matplotlib ipympl
plt.ioff()

The frequency of nth harmonic is given by
$$
f_n = n f_0 \sqrt{1 + Bn^2}
$$

In [2]:
def fn(n):
    return n*f0*(1+B*n**2)**0.5

$$
\begin{align*}
u(x,t) &= \sum_{n=1}^{\infty} \left( a_n \cos(2\pi f_n t) + b_n \sin(2\pi f_n t) \right) \sin\left(\frac{n\pi}{L}x\right)
\end{align*}
$$

In [3]:
def freq_to_u(a,b,x,t):
    u = 0
    for i in range(n_harmony):
        n = i + 1
        omega_t = 2 * np.pi * fn(n) * t
        u += a[i] * np.cos(omega_t) * np.sin(n * np.pi * x / L)
        u += b[i] * np.sin(omega_t) * np.sin(n * np.pi * x / L)
    return u

$$
\begin{align*}

a_n(t_0^+) &= a_n(t_0^-) + 2\frac{J}{L\rho} \sin\left(\frac{n\pi}{L}x_0\right) \cos(2\pi f_n t_0) \\
b_n(t_0^+) &= b_n(t_0^-) + 2\frac{J}{L\rho} \sin\left(\frac{n\pi}{L}x_0\right) \sin(2\pi f_n t_0)

\end{align*}
$$

In [4]:
def apply_impulse(a,b,x,t,J):
    '''
    This function changes a and b in place.
    '''
    for i in range(n_harmony):
        n = i + 1
        common = 2 * J / L / rho * np.sin(n * np.pi * x / L)
        omega_t = 2 * np.pi * fn(n) * t
        a[i] += common * np.cos(omega_t)
        b[i] += common * np.sin(omega_t)

Parameters

In [72]:
L = 1
tension = 1
rho = 1
ESK2 = 0.00001 # Determins stiffness. should be small.
B = np.pi**2 * ESK2 / tension / L**2
c = (tension / rho) ** 0.5
f0 = c / (2 * L)

n_harmony = 30

dt = 0.002
render_dt = 0.006

We solve for the coefficients $a_n$ and $b_n$.

In [73]:
# initial conditions

a = np.zeros(n_harmony)
b = np.zeros(n_harmony)
t=0

def update():
    global t

    # apply impulse at x = 0.3, t = 0.3
    if t <= 0.3 < t + dt:
        apply_impulse(a,b,0.3,t,0.1)

    t += dt

n_frames = 1000
rendered = []
while len(rendered) < n_frames:
    update()
    if t//render_dt > (t-dt)//render_dt:
        rendered.append(freq_to_u(a,b,np.linspace(0,L,200),t))

This animation code works in Notebook in VSCode. For other environments, you may need to change the code.

In [76]:
fig, ax = plt.subplots()
ax.set_ylim(-3, 3)

line, = ax.plot(rendered[0])

def animate(i):
    global t, render_dt
    line.set_ydata(rendered[i])
    return line,

ani = animation.FuncAnimation(fig, animate, frames=n_frames, interval=1000/30, repeat=False)
plt.show()

In [77]:
ani.save('stiffness.mp4', writer='ffmpeg', fps=30)